In [28]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [29]:
# Function to extract Product Title
def get_title(soup):
    try:
        # Outer Tag Object
        title = new_soup.find("span",attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text
        
        # Title as a string value
        title_string = title_value.strip()
    
    except AttributeError:
        title_string = ""
    
    return title_string

# Function to extract Product Price
def get_price(soup):
    try:
        price = new_soup.find("span",attrs={"class":'a-price-whole'}).text.strip()
    except AttributeError:
        price = ""
        
    return price

# Function to extract Product Rating
def get_rating(soup):
    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4'}).string.strip()
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""
    
    return rating
            
# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span",attrs={"id":'acrCustomerReviewText'}).text.strip()
        
    except AttributeError:
        review_count = ""
        
    return review_count    

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div",attrs={'id':'availability'})
        available = available.find("span").string.strip()
    
    except AttributeError:
        available = "Not Available"
        
    return available    

In [30]:
if __name__ == '__main__':
    
    # add your user agent
    HEADERS = ({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})
    
    # The webpage URL
    URL = "https://www.amazon.in/s?k=playstation5&crid=45AA12CH0P5U&sprefix=playstation%2Caps%2C305&ref=nb_sb_noss_1"
    
    # HTTP Request
    webpage = requests.get(URL,headers=HEADERS)
    
    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")
    
    # Fetch links as List of Tag Objects
    links = soup.find_all("a",attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})
    
    # Store the links
    links_list = []
    
    # Loop for extracting links from Tag Objects
    for link in links:
        links_list.append(link.get('href'))
        
    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://amazon.in" + link, headers=HEADERS)
        
        new_soup = BeautifulSoup(new_webpage.content, "html.parser")
        
        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))
        
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)    

In [31]:
amazon_df

,title,price,rating,reviews,availability
0,Sony PS5 PlayStation Console,"49,990.",3.9 out of 5 stars,145 ratings,In stock
1,Sony PS5 Digital Standalone,"39,990.",4.4 out of 5 stars,47 ratings,
2,PS5 Grand Theft Auto V,"2,099.",4.3 out of 5 stars,157 ratings,In stock
3,Call Of Duty : Modern Warfare II | Standard Ed...,"5,800.",3.8 out of 5 stars,41 ratings,Only 1 left in stock
4,Horizon Forbidden West | Standard Edition | PS...,"2,599.",4.6 out of 5 stars,265 ratings,In stock
5,PS5 Assassin's Creed Valhalla,"1,899.",4.3 out of 5 stars,152 ratings,Only 1 left in stock
6,F1 22 | Standard Edition | PS5 Game (PlayStati...,"4,249.",4.3 out of 5 stars,13 ratings,Only 2 left in stock
7,FIFA 23 | Standard Edition | PS5 (PlayStation 5),"4,855.",4.6 out of 5 stars,88 ratings,In stock
8,PS5 Mortal Kombat 11: Ultimate,"2,178.",4.4 out of 5 stars,118 ratings,
9,Business infrastructure: Business infrastructu...,,Previous page,,Not Available
